# RGB to Wavelength
Experiments on converting RGB values from PL images to wavelengths.

In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [7]:
# load the raw quantum efficiency data
# first column is wavelength in nm
# remaining columns are the %QE of the color sensor
qe_df = pd.read_csv('../frgtrainview/analysis/CS165CU_Quantum_Efficiency.csv', header=2).dropna()
qe_df

,Wavelength (nm),Blue,Green,Red
0,400.0,29.08381,5.51529,6.85800
1,405.0,33.02385,5.05232,6.01200
2,410.0,36.23982,4.56168,5.20900
3,415.0,39.10839,4.11991,4.55300
4,420.0,41.53590,3.74965,4.06200
...,...,...,...,...
116,980.0,4.57946,4.23049,4.39953
117,985.0,4.16327,3.84786,4.01697
118,990.0,3.77307,3.49821,3.67050
119,995.0,3.38447,3.15649,3.33007


In [26]:
# normalize the RGB columns by their individual min and max QE
# ie min blue QE = 0, max blue QE = 255

rgb_table = pd.DataFrame()
rgb_table['Wavelength'] = qe_df['Wavelength (nm)']

for c in ['Blue', 'Green', 'Red']:
    qe_min = qe_df[c].min()
    qe_max = qe_df[c].max()

    x = (qe_df[c] - qe_min)/(qe_max - qe_min) * 255
    rgb_table[c] = x

rgb_table

,Wavelength,Blue,Green,Red
0,400.0,134.471677,10.686089,22.005288
1,405.0,153.570296,8.788259,18.473421
2,410.0,169.159120,6.777003,15.121069
3,415.0,183.063986,4.966077,12.382410
4,420.0,194.830894,3.448289,10.332591
...,...,...,...,...
116,980.0,15.691344,5.419373,11.741706
117,985.0,13.673940,3.850877,10.144601
118,990.0,11.782517,2.417574,8.698163
119,995.0,9.898850,1.016778,7.276941


In [68]:
# determine wavelength that maps "best" to a given rgb point
# based on lowest distance between the rgb points
# TODO: maybe QE curve interpolation to get finer wavelength data?

x = (200, 100, 20)

def distance(rgb1, rgb2):
    # use squared euclidean distance
    return np.sum(np.square(rgb1 - rgb2))

closest = rgb_table[['Blue', 'Green', 'Red']].apply(lambda row: distance(x, row), axis=1).nsmallest(1)
closest.name = 'distance'
temp = rgb_table.loc[closest.index]
temp = temp.join(closest)
temp

,Wavelength,Blue,Green,Red,distance
15,475.0,235.602301,94.475048,0.818258,1665.988165


In [80]:
def rgb_to_wavelength(rgb, distance_func=distance):
    closest = rgb_table[['Red', 'Green', 'Blue']].apply(
        lambda row: distance(rgb, row),
        axis = 1
    ).nsmallest(1)

    return rgb_table['Wavelength'].loc[closest.index].values[0]

In [84]:
rgb_to_wavelength((0, 100, 200))

475.0

In [ ]:
# load an image